In [1]:
import os 
os.environ['http_proxy']="http://172.17.0.1:7890"
os.environ['https_proxy']="http://172.17.0.1:7890"
os.environ['all_proxy']="http://172.17.0.1:7890"

In [2]:
import foolbox as fb
import numpy as np
import cv2
import torch
import torchvision
from pytorchcv.model_provider import get_model as ptcv_get_model
from PIL import Image
from matplotlib import pyplot as plt
from torchvision.transforms import transforms
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
#get pretrained model
model = ptcv_get_model('resnet50',pretrained=True)

In [4]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print(size_all_mb)

97.69538116455078


In [4]:
# ImageNet datasets in pytorch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

data_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    ])

data_train = ImageFolder(root='/usr/src/app/imageNet2012/ILSVRC/Data/CLS-LOC/val',transform=data_transform)

testLoader = DataLoader(data_train, batch_size=32,shuffle=True)
# testLoader = DataLoader(data_test, batch_size=1024)

In [5]:
tempStore = torch.empty([0, 3, 224, 224])
tempLabel = torch.empty([0]).type(torch.IntTensor)
for data in testLoader:
    images,labels = data        
    for i,x in enumerate(images):
        images[i]=(x-x.min()) / (x.max()-x.min())
    tempStore = torch.cat((tempStore,images),0)
    tempLabel = torch.cat((tempLabel,labels),0)
    print(len(tempStore))
    if len(tempStore)>5000:
        torch.save(tempStore,'imgNetData-ben.pkl')
        torch.save(tempLabel,'imgNetLabel-ben.pkl')
        break
    

32
64
96
128
160
192
224
256
288
320
352
384
416
448
480
512
544
576
608
640
672
704
736
768
800
832
864
896
928
960
992
1024
1056
1088
1120
1152
1184
1216
1248
1280
1312
1344
1376
1408
1440
1472
1504
1536
1568
1600
1632
1664
1696
1728
1760
1792
1824
1856
1888
1920
1952
1984
2016
2048
2080
2112
2144
2176
2208
2240
2272
2304
2336
2368
2400
2432
2464
2496
2528
2560
2592
2624
2656
2688
2720
2752
2784
2816
2848
2880
2912
2944
2976
3008
3040
3072
3104
3136
3168
3200
3232
3264
3296
3328
3360
3392
3424
3456
3488
3520
3552
3584
3616
3648
3680
3712
3744
3776
3808
3840
3872
3904
3936
3968
4000
4032
4064
4096
4128
4160
4192
4224
4256
4288
4320
4352
4384
4416
4448
4480
4512
4544
4576
4608
4640
4672
4704
4736
4768
4800
4832
4864
4896
4928
4960
4992
5024


In [ ]:
model= model.eval()
preprocessing = dict(axis=-3)
fmodel = fb.PyTorchModel(model,bounds=(0,1),preprocessing=preprocessing)

In [ ]:
#get variable name 
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

def gen_adv_raw(model,dataLoader,fmodel,eps,atkname):
    tempStore = torch.empty([0, 3, 224, 224]).to(device)
    tempLabel = torch.empty([0]).type(torch.IntTensor).to(device)
    dataname = namestr(dataLoader,globals())[0]
    ben_acc = 0.0
    adv_acc = 0.0
    count = 0
    countStore = 2000
    idx = 0
    for data in dataLoader:
        images,labels = data
        images,labels = images.to(device),labels.to(device)
        for i,x in enumerate(images):
            images[i]=(x-x.min()) / (x.max()-x.min())
        
        ben_acc += fb.accuracy(fmodel,images,labels)
        count +=1
#         raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=eps)
        raw_advs, clipped_advs, success = attack(fmodel, images, labels,epsilons=eps)
        adv_acc += fb.accuracy(fmodel,clipped_advs,labels)
        index = success.type(torch.BoolTensor)
        cp = raw_advs[index]
        labs = labels[index]
        labs = labs.type(torch.IntTensor).to(device)
        tempStore = torch.cat((tempStore,cp),0)
        tempLabel = torch.cat((tempLabel,labs),0)
        filename_data = "advImageNetData-"+dataname+'-'+str(eps)+"-"+atkname+"-"+str(idx)+".pkl"
        filename_label = "advImageNetLabel-"+dataname+'-'+str(eps)+"-"+atkname+"-"+str(idx)+".pkl"
        if len(tempStore)>200 and countStore>0:
            print(len(tempStore))
            torch.save(tempStore,filename_data)
            torch.save(tempLabel,filename_label)
            countStore -=len(tempStore)
#             print(f'{filename_data} saved!')
            tempStore = torch.empty([0, 3, 224, 224]).to(device)
            tempLabel = torch.empty([0]).type(torch.IntTensor).to(device)
            idx+=1
        if countStore<1000:
            return ben_acc/count,adv_acc/count
            
    return ben_acc/count,adv_acc/count

In [8]:
# attack = fb.attacks.L2CarliniWagnerAttack() #L2-CW
# attack = fb.attacks.L2FastGradientAttack()    

attack = fb.attacks.L2DeepFoolAttack()      #L2-DeepFool
# attack = fb.attacks.LinfDeepFoolAttack()    #Linf-DeepFool
atkname = "L2-DeepFool"
epsilons=[0.01,0.1,0.3,0.5,1,2,3,5]
for eps in epsilons:
    ben_acc,adv_acc = gen_adv_raw(model,testLoader,fmodel,eps,atkname)
    print(f"Attack method={atkname}, epsilon={eps}, ben_acc={ben_acc},adv_acc={adv_acc}")

106
advImageNetData-testLoader-0.01-L2-DeepFool-0.pkl saved!
101
advImageNetData-testLoader-0.01-L2-DeepFool-1.pkl saved!
103
advImageNetData-testLoader-0.01-L2-DeepFool-2.pkl saved!
106
advImageNetData-testLoader-0.01-L2-DeepFool-3.pkl saved!
103
advImageNetData-testLoader-0.01-L2-DeepFool-4.pkl saved!
101
advImageNetData-testLoader-0.01-L2-DeepFool-5.pkl saved!
105
advImageNetData-testLoader-0.01-L2-DeepFool-6.pkl saved!
108
advImageNetData-testLoader-0.01-L2-DeepFool-7.pkl saved!
101
advImageNetData-testLoader-0.01-L2-DeepFool-8.pkl saved!
101
advImageNetData-testLoader-0.01-L2-DeepFool-9.pkl saved!
101
advImageNetData-testLoader-0.01-L2-DeepFool-10.pkl saved!
105
advImageNetData-testLoader-0.01-L2-DeepFool-11.pkl saved!
109
advImageNetData-testLoader-0.01-L2-DeepFool-12.pkl saved!
103
advImageNetData-testLoader-0.01-L2-DeepFool-13.pkl saved!
104
advImageNetData-testLoader-0.01-L2-DeepFool-14.pkl saved!
105
advImageNetData-testLoader-0.01-L2-DeepFool-15.pkl saved!
101
advImageNetDat

105
advImageNetData-testLoader-0.1-L2-DeepFool-11.pkl saved!
110
advImageNetData-testLoader-0.1-L2-DeepFool-12.pkl saved!
115
advImageNetData-testLoader-0.1-L2-DeepFool-13.pkl saved!
104
advImageNetData-testLoader-0.1-L2-DeepFool-14.pkl saved!
109
advImageNetData-testLoader-0.1-L2-DeepFool-15.pkl saved!
114
advImageNetData-testLoader-0.1-L2-DeepFool-16.pkl saved!
107
advImageNetData-testLoader-0.1-L2-DeepFool-17.pkl saved!
101
advImageNetData-testLoader-0.1-L2-DeepFool-18.pkl saved!
105
advImageNetData-testLoader-0.1-L2-DeepFool-19.pkl saved!
103
advImageNetData-testLoader-0.1-L2-DeepFool-20.pkl saved!
101
advImageNetData-testLoader-0.1-L2-DeepFool-21.pkl saved!
104
advImageNetData-testLoader-0.1-L2-DeepFool-22.pkl saved!
106
advImageNetData-testLoader-0.1-L2-DeepFool-23.pkl saved!
101
advImageNetData-testLoader-0.1-L2-DeepFool-24.pkl saved!
109
advImageNetData-testLoader-0.1-L2-DeepFool-25.pkl saved!
106
advImageNetData-testLoader-0.1-L2-DeepFool-26.pkl saved!
110
advImageNetData-test

112
advImageNetData-testLoader-0.1-L2-DeepFool-145.pkl saved!
102
advImageNetData-testLoader-0.1-L2-DeepFool-146.pkl saved!
106
advImageNetData-testLoader-0.1-L2-DeepFool-147.pkl saved!
101
advImageNetData-testLoader-0.1-L2-DeepFool-148.pkl saved!
109
advImageNetData-testLoader-0.1-L2-DeepFool-149.pkl saved!
109
advImageNetData-testLoader-0.1-L2-DeepFool-150.pkl saved!
104
advImageNetData-testLoader-0.1-L2-DeepFool-151.pkl saved!
101
advImageNetData-testLoader-0.1-L2-DeepFool-152.pkl saved!
109
advImageNetData-testLoader-0.1-L2-DeepFool-153.pkl saved!
102
advImageNetData-testLoader-0.1-L2-DeepFool-154.pkl saved!
107
advImageNetData-testLoader-0.1-L2-DeepFool-155.pkl saved!
109
advImageNetData-testLoader-0.1-L2-DeepFool-156.pkl saved!
112
advImageNetData-testLoader-0.1-L2-DeepFool-157.pkl saved!
101
advImageNetData-testLoader-0.1-L2-DeepFool-158.pkl saved!
113
advImageNetData-testLoader-0.1-L2-DeepFool-159.pkl saved!
101
advImageNetData-testLoader-0.1-L2-DeepFool-160.pkl saved!
103
advI

122
advImageNetData-testLoader-0.3-L2-DeepFool-97.pkl saved!
103
advImageNetData-testLoader-0.3-L2-DeepFool-98.pkl saved!
118
advImageNetData-testLoader-0.3-L2-DeepFool-99.pkl saved!
116
advImageNetData-testLoader-0.3-L2-DeepFool-100.pkl saved!
108
advImageNetData-testLoader-0.3-L2-DeepFool-101.pkl saved!
104
advImageNetData-testLoader-0.3-L2-DeepFool-102.pkl saved!
114
advImageNetData-testLoader-0.3-L2-DeepFool-103.pkl saved!
103
advImageNetData-testLoader-0.3-L2-DeepFool-104.pkl saved!
102
advImageNetData-testLoader-0.3-L2-DeepFool-105.pkl saved!
109
advImageNetData-testLoader-0.3-L2-DeepFool-106.pkl saved!
119
advImageNetData-testLoader-0.3-L2-DeepFool-107.pkl saved!
101
advImageNetData-testLoader-0.3-L2-DeepFool-108.pkl saved!
115
advImageNetData-testLoader-0.3-L2-DeepFool-109.pkl saved!
112
advImageNetData-testLoader-0.3-L2-DeepFool-110.pkl saved!
103
advImageNetData-testLoader-0.3-L2-DeepFool-111.pkl saved!
115
advImageNetData-testLoader-0.3-L2-DeepFool-112.pkl saved!
109
advImag

121
advImageNetData-testLoader-0.3-L2-DeepFool-230.pkl saved!
108
advImageNetData-testLoader-0.3-L2-DeepFool-231.pkl saved!
114
advImageNetData-testLoader-0.3-L2-DeepFool-232.pkl saved!
101
advImageNetData-testLoader-0.3-L2-DeepFool-233.pkl saved!
103
advImageNetData-testLoader-0.3-L2-DeepFool-234.pkl saved!
121
advImageNetData-testLoader-0.3-L2-DeepFool-235.pkl saved!
121
advImageNetData-testLoader-0.3-L2-DeepFool-236.pkl saved!
103
advImageNetData-testLoader-0.3-L2-DeepFool-237.pkl saved!
101
advImageNetData-testLoader-0.3-L2-DeepFool-238.pkl saved!
105
advImageNetData-testLoader-0.3-L2-DeepFool-239.pkl saved!
111
advImageNetData-testLoader-0.3-L2-DeepFool-240.pkl saved!
105
advImageNetData-testLoader-0.3-L2-DeepFool-241.pkl saved!
114
advImageNetData-testLoader-0.3-L2-DeepFool-242.pkl saved!
102
advImageNetData-testLoader-0.3-L2-DeepFool-243.pkl saved!
107
advImageNetData-testLoader-0.3-L2-DeepFool-244.pkl saved!
123
advImageNetData-testLoader-0.3-L2-DeepFool-245.pkl saved!
119
advI

...

Attack method=L2-DeepFool, epsilon=0.5, ben_acc=0.7630758157389635,adv_acc=0.21413147792706333
127
advImageNetData-testLoader-1-L2-DeepFool-0.pkl saved!
119
advImageNetData-testLoader-1-L2-DeepFool-1.pkl saved!
121
advImageNetData-testLoader-1-L2-DeepFool-2.pkl saved!
126
advImageNetData-testLoader-1-L2-DeepFool-3.pkl saved!
125
advImageNetData-testLoader-1-L2-DeepFool-4.pkl saved!
123
advImageNetData-testLoader-1-L2-DeepFool-5.pkl saved!
121
advImageNetData-testLoader-1-L2-DeepFool-6.pkl saved!
125
advImageNetData-testLoader-1-L2-DeepFool-7.pkl saved!
122
advImageNetData-testLoader-1-L2-DeepFool-8.pkl saved!
120
advImageNetData-testLoader-1-L2-DeepFool-9.pkl saved!
127
advImageNetData-testLoader-1-L2-DeepFool-10.pkl saved!
126
advImageNetData-testLoader-1-L2-DeepFool-11.pkl saved!
121
advImageNetData-testLoader-1-L2-DeepFool-12.pkl saved!
124
advImageNetData-testLoader-1-L2-DeepFool-13.pkl saved!
123
advImageNetData-testLoader-1-L2-DeepFool-14.pkl saved!
122
advImageNetData-testLoader

125
advImageNetData-testLoader-1-L2-DeepFool-137.pkl saved!
126
advImageNetData-testLoader-1-L2-DeepFool-138.pkl saved!
123
advImageNetData-testLoader-1-L2-DeepFool-139.pkl saved!
122
advImageNetData-testLoader-1-L2-DeepFool-140.pkl saved!
126
advImageNetData-testLoader-1-L2-DeepFool-141.pkl saved!
124
advImageNetData-testLoader-1-L2-DeepFool-142.pkl saved!
120
advImageNetData-testLoader-1-L2-DeepFool-143.pkl saved!
123
advImageNetData-testLoader-1-L2-DeepFool-144.pkl saved!
119
advImageNetData-testLoader-1-L2-DeepFool-145.pkl saved!
121
advImageNetData-testLoader-1-L2-DeepFool-146.pkl saved!
125
advImageNetData-testLoader-1-L2-DeepFool-147.pkl saved!
126
advImageNetData-testLoader-1-L2-DeepFool-148.pkl saved!
120
advImageNetData-testLoader-1-L2-DeepFool-149.pkl saved!
120
advImageNetData-testLoader-1-L2-DeepFool-150.pkl saved!
124
advImageNetData-testLoader-1-L2-DeepFool-151.pkl saved!
123
advImageNetData-testLoader-1-L2-DeepFool-152.pkl saved!
120
advImageNetData-testLoader-1-L2-Deep

123
advImageNetData-testLoader-1-L2-DeepFool-274.pkl saved!
120
advImageNetData-testLoader-1-L2-DeepFool-275.pkl saved!
121
advImageNetData-testLoader-1-L2-DeepFool-276.pkl saved!
127
advImageNetData-testLoader-1-L2-DeepFool-277.pkl saved!
124
advImageNetData-testLoader-1-L2-DeepFool-278.pkl saved!
122
advImageNetData-testLoader-1-L2-DeepFool-279.pkl saved!
125
advImageNetData-testLoader-1-L2-DeepFool-280.pkl saved!
124
advImageNetData-testLoader-1-L2-DeepFool-281.pkl saved!
122
advImageNetData-testLoader-1-L2-DeepFool-282.pkl saved!
121
advImageNetData-testLoader-1-L2-DeepFool-283.pkl saved!
126
advImageNetData-testLoader-1-L2-DeepFool-284.pkl saved!
122
advImageNetData-testLoader-1-L2-DeepFool-285.pkl saved!
120
advImageNetData-testLoader-1-L2-DeepFool-286.pkl saved!
124
advImageNetData-testLoader-1-L2-DeepFool-287.pkl saved!
123
advImageNetData-testLoader-1-L2-DeepFool-288.pkl saved!
122
advImageNetData-testLoader-1-L2-DeepFool-289.pkl saved!
124
advImageNetData-testLoader-1-L2-Deep

128
advImageNetData-testLoader-2-L2-DeepFool-20.pkl saved!
128
advImageNetData-testLoader-2-L2-DeepFool-21.pkl saved!
128
advImageNetData-testLoader-2-L2-DeepFool-22.pkl saved!
128
advImageNetData-testLoader-2-L2-DeepFool-23.pkl saved!
128
advImageNetData-testLoader-2-L2-DeepFool-24.pkl saved!
128
advImageNetData-testLoader-2-L2-DeepFool-25.pkl saved!
128
advImageNetData-testLoader-2-L2-DeepFool-26.pkl saved!
127
advImageNetData-testLoader-2-L2-DeepFool-27.pkl saved!
128
advImageNetData-testLoader-2-L2-DeepFool-28.pkl saved!
127
advImageNetData-testLoader-2-L2-DeepFool-29.pkl saved!
128
advImageNetData-testLoader-2-L2-DeepFool-30.pkl saved!
128
advImageNetData-testLoader-2-L2-DeepFool-31.pkl saved!
128
advImageNetData-testLoader-2-L2-DeepFool-32.pkl saved!
128
advImageNetData-testLoader-2-L2-DeepFool-33.pkl saved!
128
advImageNetData-testLoader-2-L2-DeepFool-34.pkl saved!
126
advImageNetData-testLoader-2-L2-DeepFool-35.pkl saved!
128
advImageNetData-testLoader-2-L2-DeepFool-36.pkl save

KeyboardInterrupt: 

In [11]:
# attack = fb.attacks.L2CarliniWagnerAttack() #L2-CW
# attack = fb.attacks.L2FastGradientAttack()    

# attack = fb.attacks.L2DeepFoolAttack()      #L2-DeepFool
attack = fb.attacks.LinfDeepFoolAttack()    #Linf-DeepFool
atkname = "Linf-DeepFool"
epsilons=[0.01,0.1,0.15,0.3,0.5,1,3,5]
for eps in epsilons:
    ben_acc,adv_acc = gen_adv_raw(model,testLoader,fmodel,eps,atkname)
    print(f"Attack method={atkname}, epsilon={eps}, ben_acc={ben_acc},adv_acc={adv_acc}")

223
224
223
223
224
224
223
222
223
221
222
224
222
223
222
223
222
224
223
222
223
222
223
223
223
223
222
223
222
222
223
223
223
223
223
223
223
223
223
224
221
221
223
223
224
224
224
223
224
223
223
222
223
224
223
221
224
223
223
223
224
223
224
221
222
223
224
224
222
224
224
223
223
223
222
223
224
224
223
223
224
224
224
224
224
223
223
222
224
224
222
222
224
222
223
223
224
224
224
223
223
223
224
222
224
224
224
223
223
222
222
224
223
223
222
224
223
222
223
223
222
223
223
224
222
223
223
224
221
224
224
224
224
224
223
223
224
223
223
223
223
223
224
222
222
222
222
223
224
223
223
220
222
223
223
224
223
221
224
223
223
224
221
224
224
222
224
224
224
223
222
221
224
224
223
222
224
223
223
223
223
222
223
223
224
223
222
223
224
220
223
223
223
221
223
223
222
223
223
223
224
224
224
224
224
224
223
224
223
224
222
224
222
224
223
224
223
223
223
224
224
224
224
Attack method=Linf-DeepFool, epsilon=0.01, ben_acc=0.7629758477287268,adv_acc=0.004238643634037108
224
224
2

In [12]:
# attack = fb.attacks.L2CarliniWagnerAttack() #L2-CW
attack = fb.attacks.L2FastGradientAttack()    

# attack = fb.attacks.L2DeepFoolAttack()      #L2-DeepFool
# attack = fb.attacks.LinfDeepFoolAttack()    #Linf-DeepFool
atkname = "L2FGSM"
epsilons=[0.01,0.1,0.3,0.5,1,2,3,5]
for eps in epsilons:
    ben_acc,adv_acc = gen_adv_raw(model,testLoader,fmodel,eps,atkname)
    print(f"Attack method={atkname}, epsilon={eps}, ben_acc={ben_acc},adv_acc={adv_acc}")

204
201
202
212
203
205
201
201
207
208
201
210
204
201
206
201
207
207
212
209
210
201
202
203
210
202
205
201
201
202
202
205
202
202
205
207
201
204
213
208
213
209
205
202
204
201
204
203
201
204
206
203
215
202
205
206
205
205
207
209
Attack method=L2FGSM, epsilon=0.01, ben_acc=0.7630758157389635,adv_acc=0.7505598208573256
206
211
209
206
208
213
202
213
206
208
201
206
201
208
206
204
202
206
215
201
201
210
209
207
205
201
210
208
205
210
201
202
205
208
207
208
208
203
204
212
203
207
207
210
204
201
209
204
207
204
209
209
204
207
206
207
202
210
210
204
206
208
211
205
203
210
203
206
203
209
208
201
201
202
210
206
210
206
205
201
211
208
206
206
210
208
Attack method=L2FGSM, epsilon=0.1, ben_acc=0.7630758157389635,adv_acc=0.644193857965451
205
211
218
205
209
210
211
207
221
213
214
211
202
215
208
206
207
206
211
201
218
212
220
204
208
208
209
208
204
201
211
210
210
209
203
211
207
204
207
214
207
206
205
212
217
206
205
213
205
216
215
215
208
211
213
201
202
204
215
20

In [9]:
attack = fb.attacks.L2CarliniWagnerAttack(binary_search_steps=9, steps=6000, stepsize=0.01, confidence=0.0, initial_const=0.001, abort_early=True) #L2-CW
# attack = fb.attacks.L2FastGradientAttack()    
# attack = fb.attacks.L2DeepFoolAttack()      #L2-DeepFool
# attack = fb.attacks.LinfDeepFoolAttack()    #Linf-DeepFool
atkname = "L2-CW"
epsilons=[0.3]
for eps in epsilons:
    ben_acc,adv_acc = gen_adv_raw(model,testLoader,fmodel,eps,atkname)
    print(f"Attack method={atkname}, epsilon={eps}, ben_acc={ben_acc},adv_acc={adv_acc}")

202
209
211
207
207
215
212
222
213
206


KeyboardInterrupt: 

In [7]:
attack = fb.attacks.L2CarliniWagnerAttack(binary_search_steps=9, steps=10000, stepsize=0.01, confidence=0.4, initial_const=0.001, abort_early=True) #L2-CW
# attack = fb.attacks.L2FastGradientAttack()    
# attack = fb.attacks.L2DeepFoolAttack()      #L2-DeepFool
# attack = fb.attacks.LinfDeepFoolAttack()    #Linf-DeepFool
atkname = "L2-CW"
epsilons=[0.3]
for eps in epsilons:
    ben_acc,adv_acc = gen_adv_raw(model,testLoader,fmodel,eps,atkname)
    print(f"Attack method={atkname}, epsilon={eps}, ben_acc={ben_acc},adv_acc={adv_acc}")

209
211
211
218
213
Attack method=L2-CW, epsilon=0.3, ben_acc=0.7677083333333333,adv_acc=0.446875


In [ ]:
attack = fb.attacks.L2CarliniWagnerAttack(binary_search_steps=9, steps=10000, stepsize=0.01, confidence=1.0, initial_const=0.001, abort_early=True) #L2-CW
# attack = fb.attacks.L2FastGradientAttack()    
# attack = fb.attacks.L2DeepFoolAttack()      #L2-DeepFool
# attack = fb.attacks.LinfDeepFoolAttack()    #Linf-DeepFool
atkname = "L2-CW"
epsilons=[0.3]
for eps in epsilons:
    ben_acc,adv_acc = gen_adv_raw(model,testLoader,fmodel,eps,atkname)
    print(f"Attack method={atkname}, epsilon={eps}, ben_acc={ben_acc},adv_acc={adv_acc}")

In [261]:

img_adv1 = copy.deepcopy(advs_[0])

In [211]:
def get_single_mask(num):
    x_b = np.zeros((32,32))

    for i in range(num):
        x_axis = np.random.randint(1, 31)
        y_axis = np.random.randint(1, 31)
        case_radom = np.random.randint(1, 5)
        
        if (case_radom == 1) :
            x_b[x_axis][y_axis] = 1
        
        elif (case_radom == 2) :
            x_b[x_axis][y_axis+1] = 1
        
        elif (case_radom == 3) :
            x_b[x_axis+1][y_axis] = 1
        
        elif (case_radom == 4) :
            x_b[x_axis+1][y_axis+1] = 1
    
#     x_b_msk = np.img_as_ubyte(x_b)
    return x_b

In [212]:
try_times = 11
def get_masks(num):

    mask = np.expand_dims(get_single_mask(num), axis = 0)
    for k in range(try_times-1):
        mask2 = np.expand_dims(get_single_mask(num), axis = 0)
        mask = np.concatenate((mask, mask2))
    
    # print(mask.shape) ->  (11, 224, 224)
    return mask

In [272]:
# mask = get_masks(300)
dst_adv.shape

(32, 32, 3)